# NLP project


Project 21: Automatic Summarization  

We shall consider structured document containing a title, abstract and a set of subsections. We would like to build a text summarizer such that tracks important keywords in the document. For this purpose, the first step is identify these keywords.  

In [ ]:
pip install --upgrade pip

In [ ]:
!pip list
# tarkista löytyykö: lxml, html5lib, requests, selenium, webdriver-manager
# lisäohjeita task1

In [ ]:
#jos nltk ei löydy asenna -> ! pip install nltk
import nltk
nltk.download("stopwords")
#from nltk.cluster.util import cosine_distance

## TASK 1
Assume the initial input is given as html document (choose an example of your own), we hypothesize that important keywords are initially contained in the words of titles, abstract and possibly titles of subsections of the document. Suggest a simple python script that inputs an html document and outputs the lists of words in the title, abstract and title of section/subsections.

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from nltk.tokenize import sent_tokenize
import time
# Kaikki sivut ei anna koko html bodyä käyttämällä pelkkää requestia. Seleniumilla näyttää toimivan useammilla. 
# pip install -U selenium
# pip install webdriver-manager
# jos käytät anacondaa eikä meinaa toimia niin kokeile myös $ conda update pip

# Collect title, subtitles, abstract and body text from html file.
# Print out titles and abstract and construct one string based on
# the elements.

def _convertHtmlToStr(elements):
    str = ""
    for element in elements:
        if len(element.text.split()) > 1:
            str += element.text
            if not str.endswith("."):
                str += "."
            str += " "
    sentences = sent_tokenize(str)
    return str, len(sentences)

url = "https://ieeexplore.ieee.org/document/6809191"
article = ""
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)

# Wait for article to fully load
time.sleep(3)

soup = BeautifulSoup(driver.page_source, 'lxml')
strElement = ""
countTitle, countAbstract, countH2, countH3, countH4, countP = 0, 0, 0, 0, 0, 0

strElement, countTitle = _convertHtmlToStr(soup.find("h1", {"class": "document-title"}))
print("Title:\n{}\n\n".format(strElement))
article += strElement
article += ". "
strElement, countAbstract = _convertHtmlToStr(soup.find("div", {"class": "abstract-text"}))
print("Abstract:\n{}\n\n".format(strElement))
article += strElement

articleHtmlBody = soup.find("div", {"id": "article"})
if articleHtmlBody == None:
    raise ValueError

strElement, countH2 = _convertHtmlToStr(articleHtmlBody.find_all("h2"))
print("Section titles:\n{}\n\n".format(strElement))
article += strElement
strElement, countH3 = _convertHtmlToStr(articleHtmlBody.find_all("h3"))
print("Subsection titles:\n{}\n\n".format(strElement))
article += strElement
strElement, countH4 = _convertHtmlToStr(articleHtmlBody.find_all("h4"))
print("Subsubsection titles:\n{}\n\n".format(strElement))
article += strElement
strElement, countP = _convertHtmlToStr(articleHtmlBody.find_all("p"))
article += strElement
countP += 1

driver.close()

counts = [countTitle, countAbstract, countH2, countH3, countH4, countP]

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\Dark\.wdm\drivers\chromedriver\win32\95.0.4638.54\chromedriver.exe] found in cache
C:\Users\Dark\anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


AttributeError: 'Comment' object has no attribute 'text'

In [1]:
import yake

In [2]:
#Keyword search and analysis

w_extractor = yake.KeywordExtractor()

language = "en"
max_ngram_size = 2
deduplication_threshold = 0.9
numOfKeywords = 50 #alunperin 10

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(article)

for kw in keywords:
    print(kw)


NameError: name 'article' is not defined

## TASK 2
Write down a simple python script that allows you to output the histogram of word frequency in the document, excluding the stopwords (see examples in online NLTK book). Use SpaCy named-entity tagger to identify person-named entities and organization-named entities in the document.

In [8]:
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import numpy as np
from nltk.tokenize import word_tokenize
from collections import Counter

all_stopwords = stopwords.words('english')
all_stopwords.append('The')

In [9]:
text_tokens = word_tokenize(article)
tokens_without_sw = [word for word in text_tokens if word.isalpha() and word not in all_stopwords]

print(tokens_without_sw)
#print(filtered_sentence)

[]


In [10]:

### Count histogram from every word manually ###
#charsToRemove = ".,()"
#wordCounts = Counter(tokens_without_sw)
wordCounts = Counter(tokens_without_sw)
wordCounts = wordCounts.most_common()

print(wordCounts)

wordCounts = wordCounts[0:20]

words = list(zip(*wordCounts))[0]
occurency = list(zip(*wordCounts))[1]
fig, ax = plt.subplots(figsize=(18,5))
plt.bar(np.arange(len(words)), occurency, align='center')
plt.xticks(np.arange(len(words)), words, rotation='vertical')
plt.ylabel('Keyword count')
plt.xlabel('Keyword id')
plt.show()

[]


IndexError: list index out of range

In [11]:
#Use SpaCy to identify person-named entities and organization-named entities
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

#vinkkiä https://towardsdatascience.com/named-entity-recognition-with-nltk-and-spacy-8c4a7d88e7da

#Identifying person and organization-named entities
wordsInStr = ""
for word in tokens_without_sw:
    wordsInStr += word
    wordsInStr += " "
    
nlp = en_core_web_sm.load()
doc = nlp(wordsInStr)

#Print only ORG or PERSON labeled entities
if doc.ents:
    for ent in doc.ents:
        if ent.label_ == "ORG" or ent.label_ == "PERSON":
            print(ent.text+ " - " + ent.label_)
else:
    print("No named entities found.")

No named entities found.


## TASK 3

We would like the summarizer to contain frequent wording (excluding stopwords) and as many named-entities as possible. For this purpose, use the following heuristic to construct the summarizer. First we shall assume each sentence of the document as individual sub-document. Use TfIdf vectorizer to output the individual tfidef score of each word of each sentence (after initial preprocessing and wordnet lemmatization stage). Then consider only sentences that contain person or organization named-entities and use similar approach to output the tfidf score of the named-entities in each sentence. Finally construct the sentence (S) weight as a  weighted sum:
<br>
$$S_{weight}=\sum_{w\varepsilon S}W_{TfiDf}+2\sum_{NM\varepsilon S}NM_{TfiDf}+POS_s$$
<br>
where NMTfiDF stands for the TfIdF of named-entity NM in sentence S.  POSS corresponds to the sentence weight associated to the location of the sentence. So that the sentence location weight will be maximum (1) if located in the title of the document, 0.5 if located  in the title of one of the subsection, 0.25 if located in the title one of the subsubsection, 0.1 if located in one representative object of the document, and 0 if located only in the main text. Make sure to normalize the term tfidf and Nm tfidf weights and suggest a script to implement the preceding accordingly, so that the summarizer will contain the 10 sentences with the highest Sweight scores.  


In [12]:
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

NameError: name 'nltk' is not defined

In [ ]:
weights = {
    "1": 1,
    "2": 0.1,
    "3": 0.5,
    "4": 0.25,
    "else": 0.0
}

In [ ]:
def _calculateFullScores(sentenceScores, namedEntityScores, counts):
    scaler = MinMaxScaler()
    weightList= []

    if len(counts) > 0:
        if counts[2] == 0:
            counts.pop[2]
    else:
        counts = [0, 0, 0, 0, len(sentenceScores)]

    for i in range(len(counts)):
        for j in range(counts[i]):
            if i > 3:
                weightList.append(weights["else"])
            else:
                weightList.append(weights[str(i+1)])

    df = pd.DataFrame({
        "Weights": weightList,
        "SentenceScores": sentenceScores,
        "EntityScores": namedEntityScores,
    })

    df[["SentencesScaled"]] = scaler.fit_transform(df[["SentenceScores"]])
    df[["EntitiesScaled"]] = scaler.fit_transform(df[["EntityScores"]])
    df["S_weight"] = df["SentencesScaled"] + (2 * df["EntitiesScaled"]) + df["Weights"]

    return df["S_weight"].tolist()


def _getNamedEntities(article):
    nlp = en_core_web_sm.load()
    doc = nlp(article)
    namedEntities = []
    
    for ent in doc.ents:
        if ent.label_ == "ORG" or ent.label_ == "PERSON":
                namedEntities.append(ent.text)

    return namedEntities


def _getSentencesWithMaxWeights(weights, sentences, numberOfSentences):
    arr = np.array(weights)
    indexes = np.argpartition(arr, -numberOfSentences)[-numberOfSentences:]
    sentences = np.array(sentences)
    return sentences[indexes]


def _preProcess(document):
    stopwords = list(set(nltk.corpus.stopwords.words('english')))
    WN_lemmatizer = WordNetLemmatizer()
    sentences = sent_tokenize(document)
    processedSentences = []
    tokens = []

    for sentence in sentences:
        words = word_tokenize(sentence)
        words = [WN_lemmatizer.lemmatize(word, pos="v") for word in words]

        # get rid of numbers and Stopwords
        words = [word for word in words if word.isalpha() and word not in stopwords]
        processedSentences.append(' '.join(word for word in words))
        tokens.extend(words)

    return processedSentences, tokens


def _tfidfScores(corpus, sentences):
    tfidf = TfidfVectorizer()
    fittedVectorizer = tfidf.fit(corpus)
    vectors = fittedVectorizer.transform(sentences).toarray()

    scores = []
    for i in range(len(vectors)):
        score = 0
        for j in range(len(vectors[i])):
            score = score + vectors[i][j]

        scores.append(score)
    return scores

In [ ]:
def findTopSentences(document, numberOfSentences, isUrl):
    sentences, tokens = _preProcess(document)
    sentenceTfidfScores = _tfidfScores(tokens, sentences)
    namedEntitiesTfidfScores = _tfidfScores(_getNamedEntities(document), sentences)
    if isUrl:
        SWeight = _calculateFullScores(sentenceTfidfScores, namedEntitiesTfidfScores, counts)
    else:
        SWeight = _calculateFullScores(sentenceTfidfScores, namedEntitiesTfidfScores, [])
    topSentences = _getSentencesWithMaxWeights(SWeight, sent_tokenize(document), numberOfSentences)
    return topSentences

topSentences = findTopSentences(article, 10, True)
for sentence in topSentences:
    print("{}\n".format(sentence))


## TASK 4
Test the above approach with Opinosis dataset available at https://kavita-ganesan.com/opinosis-opinion-dataset/#.YVw6J5ozY2x,  and record the corresponding Rouge-2 and Rouge-3 evaluation score. 

In [ ]:
#Rouge 2 ja 3 scoring
https://towardsdatascience.com/the-ultimate-performance-metric-in-nlp-111df6c64460


In [ ]:
#pip install git+git://github.com/bdusell/rougescore.git

'''
In ROUGE, a "peer" summary produced by a machine summarization system is compared against 
one or more hand-written "model" summaries and then assigned a score from 0 to 1. This score is the
F-measure of recall vs. precision, and the evaluator can adjust a parameter α to control whether this 
score favors recall (does the peer summary contain all of the information in the model summaries?) 
or precision (does the peer summary contain only information in the model summaries?). 

When α ≈ 0, this score favors recall; when α ≈ 1, it favors precision. 
In the DUC conferences, α was set to 0, and a hard length limit was imposed on generated summaries. 
The original ROUGE implementation uses α = 0.5 by default.
'''



In [ ]:
import rougescore as rouge

def rouge(peer, model):
    rougeBi = rouge.rouge_2(peer, model, 1)
    print("Rouge 2: ", rougeBi)
    rougeTri = rouge.rouge_3(peer, model, 1)
    print("Rouge 3: ", rougeTri)
    
    return rougeBi,rougeTri

In [ ]:
def read_folder(dir):
    
    topic = []
    
    for file in os.listdir(dir):   
        with open(os.path.join(dir + file)) as f:
            doc = f.readlines()
            f.close()
            doc = " ".join(doc)
            topic.append(doc)
            
    return topic           

def create_model(dir):
    
    model = []
    
    for folder in os.listdir(directory + "summaries-gold/"):
        
        gold = read_folder(directory + "summaries-gold/" + folder + "/")
        model.append(gold)
        
    
    return model

In [ ]:
def summary(dir):
    
    list_summary = []
    for file in os.listdir(directory + "topics/"):
            
        with open(os.path.join(dir + "topics/" + file)) as f:
            doc = f.readlines()
            f.close()
            doc = " ".join(doc)
            
            summary = list(findTopSentences(doc, 10, False))
            list_summary.append(summary)
            
    return list_summary


In [ ]:
import os

directory = "C:/Users/Dark/Downloads/data/Opinosis_dataset/"

summary = summary(directory)
model = create_model(directory)

In [ ]:
list_score = []

for i in range(len(summary)):
    bi, tri = rouge(summary, model)
    list_score.append((bi,tri))

In [ ]:
print(list_score)

## TASK 5

[x] We would like to improve the summarization by taking into account the diversity among the sentence in the sense that we would like to minimize redundancy among sentences. For this purpose, we shall use the sentence-to-sentence semantic similarity introduced in the NLP lab. 

[x] Next, instead of recording only the 10 sentences with highest Sweight scores, we shall record the 20 top sentences in terms of $S_{weight}$ scores. Then the selection of the top 10 sentences among the 20 sentences follows the following approach. 

[x] First, order the 20 sentences in the decreasing order of their $S_{weight}$ scores, say S1, S2, …, S20 (where S1 is the top ranked and S20 the 20th ranked sentence). 

[x] Second, we shall assume that S1 is always included in the summarizer, we shall then attempt to find the other sentences among S2 till S20 to be included into the summarizer. 

[x] Calculate the sentence-to-sentence similarity Sim(S1,Si) for i=1 to 20, the Sentence Sj that yields the minimum similarity with S1 will therefore be included in the summarizer. 

[x] Next, for each of the remaining sentences Sk (with k different from 1 and j), we calculate the sentence similarity with Sj. Therefore the sentence Sp that yields minimum value of “Sim(Sp, S1)+Sim(Sp,Sj)” will be included in the summarizer (Note: the quantity Sim(Sp, S1) is already calculated in previous step).  

[x] Similarly in the next phase, we should select a sentence Sl (l different from 1, j and k) so that  “Sim(Sl, S1)+Sim(Sl,Sj)+Sim(Sl,Sp)”, Etc.. 

[x] You then stop once you reached 10 sentences included in the summarizer. 

[ ] Suggest a script that includes this process.. and illustrate its functioning in the example you chosen in 1).

In [ ]:
kirjoitan tähän itselleni että pysyn ohjeiden perässä
1.Luo 20 lauseen lista, missä lauseiden s(weight) pisteet ovat suurimmat (s1,s2,s3,...,s20)
2.s1 on tiivistelmän ensimmäinen lause 
    2.1 poista s1 listalta
3.Vertaa loppuja lauseita s1. Lause joka on vähiten samanlainen s1 kanssa lisätään tiivistelmään, ja kutsutaan s(j)
    3.1 poista s(j) listalta
4.Vertaa loppuja lauseita s(j) ja taas alin arvo lisätään tiivistelmään. Lisätty lause s(p)
    4.1 poista lause

In [ ]:
print(len(sentences))

In [ ]:
#download larger pipeline package for spaCy
python -m spacy download en_core_web_lg #tarkempi mutta 770mb kokoinen

python -m spacy download en_core_web_sm #paljon pienempi mutta ei yhtä tarkka

In [ ]:
#s1 määritys
picked_sentences = []

#choose dictionary
nlp = spacy.load("en_core_web_lg")
#nlp = spacy.load("en_core_web_md")

#löydä ensimmäinen lause, korkein s(weigth)
for sentence in sentences: 
    #lisää koodi s(weight) laskemiseen, tai valitse ensimmäinen lause jos lista on järjestyksessä
    s1 = sentence

#poista valinta listasta ja lisää tiivistelmä listaan    
picked_sentences.append(s1)
sentences.remove(s1)

print(picked_sentences)

In [ ]:
#Loppujen yhdeksän lauseen valinta

#lista samanlaisuus pisteistä
sim_score = []

#while pyörii kunnes 10 lausetta on löydetty
while(len(picked_sentences)<10):
    sim_score.clear()
    
    for sentence in sentences:
        nlp_sentence = nlp(sentence)
        score = 0

        for p_sentence in picked_sentences:
            #vertaa kahta lausetta
            nlp_p_sentence = nlp(p_sentence)

            score += nlp_p_sentence.similarity(nlp_sentence)
                
        sim_score.append(score)
        
        
    print(sim_score)
    min_value = min(sim_score)
    min_index = sim_score.index(min_value)   

    print("Sentences left in the list: " + str(len(sentences)))
    print("Smallest value: " + str(min_value))
    print(sentences[min_index])

    picked_sentences.append(sentences[min_index])
    sentences.remove(sentences[min_index])



In [ ]:
print("Summarized text")
print(picked_sentences)

## TASK 6

We would like to make the choice of keywords not based on histogram frequency but using the open source RAKE https://www.airpair.com/nlp/keyword-extraction-tutorial. Repeat the previous process of selecting the sentences that are associated to the ten first keywords generated by RAKE. Comment on the quality of this summarizer based on your observation

In [ ]:
#Repossa ollut asennus tiedosto ei kyennyt asentumaan windows ympäristössä ilman korjausta
git clone https://github.com/zelandiya/RAKE-tutorial
cd RAKE-tutorial

#Ennen asennusta mene setup.py tiedostoon ja poista slash (/) poluista: 
#package_dir={'nlp_rake': './'} ja 
#package_data={'nlp_rake': ['data/']}

#muutin "nlp-rake" nimen pelkäksi "rake" asennus tiedostossa.

#kuva setup_korjaus löytyy githubista, jonka jälkeen paketin asennus toimii
python setup.py install 



In [ ]:
#Asensin moduulin eri paikkaan kuin missä jupyter serveri polku, korjasin tällä polun
import sys 
sys.path.append("C:/NLP/RAKE-tutorial")

In [ ]:
import rake 
import operator

In [ ]:
#Korjaa polku, tiedosto löytyy githubista
rake_object = rake("C:/NLP/RAKE-tutorial/data/stoplists/SmartStoplist.txt", 5, 3, 4) 

In [ ]:
sample_file = open("C:/NLP/RAKE-tutorial/data/docs/fao_test/w2167e.txt", 'r') #aseta teksti minkä haluat käsitellä
text = sample_file.read()
sentenceList = rake.split_sentences(text)
print(sentenceList[0:1])


In [ ]:
keywords = rake_object.run(text)
#print("Keywords:", keywords[0:10]) #10 ensimmäistä
keywords_topten = []

for i in range(10):
    keywords_topten.append(keywords[i][0])
    
print(keywords_topten)

In [ ]:
#extract sentences using keywords
dct = {}
for sentence in sentenceList:
    dct[sentence] = sum(1 for word in keywords_topten if word in sentence)

rake_sentences = [key for key,value in dct.items() if value == max(dct.values())]


print("\n".join(rake_sentences))

In [ ]:
#Comparing results
print("Sentences in original text: {}, summarized amount: {}".format(len(sentenceList),len(rake_sentences)))

In [ ]:
Comment on results:

## TASK 7

It is also suggested to explore alternative implementations with larger number of summarization approaches implemented- https://github.com/miso-belica/sumy. Show how each of the implemented summarizer behaves when inputted with the same document you used in previous case.

In [ ]:
#https://github.com/miso-belica/sumy
#pip install sumy

In [ ]:
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as LSASummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer as LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer as LuhnSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

LANGUAGE = "english"
SENTENCES_COUNT = 10

In [ ]:
def sumySummarize(article):
    stemmer = Stemmer(LANGUAGE)
    summarizers = [LexRankSummarizer(stemmer), LSASummarizer(stemmer), LuhnSummarizer(stemmer)]
    parser = PlaintextParser.from_string(article, Tokenizer(LANGUAGE))
    results = []
    
    for summarizer in summarizers:
        summarizer.stop_words = get_stop_words(LANGUAGE)
        sentences = []
        for sentence in summarizer(parser.document, SENTENCES_COUNT):
            sentences.append(sentence)
        results.append(sentences)
    
    return results

sumySentences = sumySummarize(article)
for sentences in sumySentences:
    print("{}\n\n".format(sentences))

## TASK 8

Now we would like to compare the above summarizers and those in 3), 5) and 7) on a new dataset constructed as follows. First select an Elsevier journal of your own and select 10 papers highly ranked in the journal according to citation index (The journal papers should be well structured to contain Abstract, Introduction and Conclusion). 

For each of the ten papers, consider the introduction as the main document to seek to apply summarizer, and consider the Abstract and Conclusion as two golden summary of the document that you can use for assessment using ROUGE-1 and ROUGE-2 evaluation. 

Report in a table the evaluation score of each summarizer. 

In [ ]:
#Rouge 1 & 2 pisteytyts koodi


## TASK 9

Design a simple GUI that allows the user to input a text or a link to a document to be summarized and output the summarizer according to 3), algorithms implemented in 7)

In [ ]:
# run simpleGUI.py